In [28]:
from pathlib import Path

import cv2
import numpy as np

from src.dino.opencv import arcmin2_to_pixel2, read_image, write_image
from src.dino.util import generate_scales
from src.tone_mappers.util import rgb_to_xyz, xyz_to_lxy, lxy_to_rgb


def gaussian_blur(
    image: np.ndarray, sigma: float, kernel_sigmas: int = 2
) -> np.ndarray:
    # Ensure the kernel size is odd
    kernel_size = int(2 * kernel_sigmas * sigma + 1) | 1
    blurred = cv2.GaussianBlur(
        image,
        (kernel_size, kernel_size),
        sigmaX=sigma,
        sigmaY=sigma,
        borderType=cv2.BORDER_REFLECT_101,
    )
    return np.squeeze(blurred)


def generate_mipmap(
    image: np.ndarray, downscale_ratio: float, levels: int
) -> list[np.ndarray]:
    images = [image]
    for i in range(levels):
        prev_image = images[-1]
        height, width = prev_image.shape[:2]
        new_w = max(1, round(width / downscale_ratio))
        new_h = max(1, round(height / downscale_ratio))
        images.append(
            cv2.resize(prev_image, (new_w, new_h), interpolation=cv2.INTER_AREA)
        )
    return images


def efficient_blur_from_mipmap(mipmap: list[np.ndarray], level: int, sigma: int) -> np.ndarray:
    height, width = mipmap[0].shape[:2]
    blurred = gaussian_blur(mipmap[level], sigma)
    return cv2.resize(blurred, (width, height), interpolation=cv2.INTER_LINEAR)


def bronto_efficient(
    rgb_image: np.ndarray,
    cs_ratio: float = 2.0,
    num_scales: int = 13,
    k: float = 0.3,
    w: float = 0.9,
    m: float = 0.1,
    d_nit_arcmin2: float = 100,
    image_fov_degrees: float = 72,
) -> np.ndarray:
    X, Y, Z = rgb_to_xyz(rgb_image)
    L, x_chroma, y_chroma = xyz_to_lxy(X, Y, Z)
    width = L.shape[1]
    d = arcmin2_to_pixel2(d_nit_arcmin2, width, image_fov_degrees)

    mipmap = generate_mipmap(L, cs_ratio, num_scales)
    num_scales = len(mipmap)
    weights = [w**i for i in range(num_scales)]
    scales = generate_scales(width, cs_ratio, num_scales)
    center_response = efficient_blur_from_mipmap(mipmap, 0, cs_ratio)
    accum = np.zeros_like(L)
    c_sum = np.zeros_like(L)

    for i in range(1, num_scales):
        surround_response = efficient_blur_from_mipmap(mipmap, i, cs_ratio)
        w = weights[i - 1]
        _d = d / scales[i] ** 2
        c = w * np.abs((center_response + _d) / (surround_response + _d) - 1) + m
        accum += c * surround_response
        c_sum += c
        center_response = surround_response

    local_white = accum / c_sum
    L_tonemapped = (k / local_white) * L
    return lxy_to_rgb(L_tonemapped, x_chroma, y_chroma)


def bronto(
    rgb_image: np.ndarray,
    cs_ratio: float = 2.0,
    num_scales: int = 13,
    k: float = 0.3,
    w: float = 0.9,
    m: float = 0.1,
    d_nit_arcmin2: float = 100,
    image_fov_degrees: float = 72,
) -> np.ndarray:
    X, Y, Z = rgb_to_xyz(rgb_image)
    L, x_chroma, y_chroma = xyz_to_lxy(X, Y, Z)
    width = L.shape[1]
    d = arcmin2_to_pixel2(d_nit_arcmin2, width, image_fov_degrees)

    scales = generate_scales(width, cs_ratio, num_scales)
    weights = [w**i for i in range(len(scales))]
    center_response = gaussian_blur(L, scales[0])
    accum = np.zeros_like(L)
    c_sum = np.zeros_like(L)

    for i in range(1, len(scales)):
        surround_response = gaussian_blur(L, scales[i])
        w = weights[i - 1]
        _d = d / scales[i] ** 2
        c = w * np.abs((center_response + _d) / (surround_response + _d) - 1) + m
        accum += c * surround_response
        c_sum += c
        center_response = surround_response

    local_white = accum / c_sum
    L_tonemapped = (k / local_white) * L
    return lxy_to_rgb(L_tonemapped, x_chroma, y_chroma)


def tonemap_image(image_path, output_path, tone_mapper, name, *args, **kwargs):
    image = read_image(str(image_path))
    tonemapped = tone_mapper(image, *args, **kwargs)
    file_path = f"{output_path / image_path.stem}_{name}.png"
    write_image(file_path, tonemapped)
    print("Exported", file_path)

In [31]:
INPUT_IMAGE = Path("images/LuxoDoubleChecker.exr")
OUTPUT_PATH = Path("_output")

In [26]:
tonemap_image(INPUT_IMAGE, OUTPUT_PATH, bronto, "bronto")

Exported _output/HancockKitchenInside_bronto.png


In [32]:
tonemap_image(INPUT_IMAGE, OUTPUT_PATH, bronto_efficient, "bronto_efficient_linear")

Exported _output/LuxoDoubleChecker_bronto_efficient_linear.png


In [11]:
mipmap = generate_mipmap(read_image(str(INPUT_IMAGE)), 2, 10)
blurs = [efficient_blur_from_mipmap(mipmap, i, 2) for i in range(len(mipmap))]
for i, image in enumerate(blurs):
    write_image(f"{OUTPUT_PATH / INPUT_IMAGE.stem}_level{i}.png", image)